In [2]:
import numpy as np
import cvxpy as cp
from scipy.io import loadmat
from rsome import ro
from rsome import grb_solver as grb

# Intervalo entorno a la media

Importar datos del problema

In [3]:
data = loadmat("Datos_Manufacturing_media.mat")
A_media = data['A_media']
A_hat_media = data['A_hat_media']
b = data['b'].flatten()

c = np.array([1.0,1.0,1.0,1.0,1.0]) 

In [4]:
# === Parámetros del problema ===
m, n = A_media.shape

# Todos los coeficientes son inciertos
J = [list(range(n)) for _ in range(m)]

# === Probar con distintos valores de Gamma ===
for Gamma_value in [1.0, 2.0, 3.0, 4.0, 5.0]:
    print(f"\n--- Resolviendo con Γ = {Gamma_value} ---")

    Gamma = np.array([Gamma_value] * m)
    model = ro.Model()

    x = model.dvar(n)
    y = model.dvar(n)
    z = model.dvar(m)
    p = {}

    # Declarar variables auxiliares p_ij
    for i in range(m):
        for j in J[i]:
            p[i, j] = model.dvar()

    # Restricciones robustas
    for i in range(m):
        expr = sum(A_media[i, j] * x[j] for j in range(n)) + Gamma[i] * z[i] + sum(p[i, j] for j in J[i])
        model.st(expr <= b[i])

        for j in J[i]:
            model.st(z[i] + p[i, j] >= A_hat_media[i, j] * y[j])
            model.st(p[i, j] >= 0)

    # Relación y_j = |x_j|
    for j in range(n):
        model.st(x[j] <= y[j])
        model.st(-x[j] <= y[j])
        model.st(y[j] >= 0)

    # Dominio de x
    model.st(x >= 0)

    # Objetivo
    model.max(c @ x)

    # Resolver
    model.solve(grb)

    # Resultados
    x_val = x.get()
    obj_val = model.get()
    print(f"Solución x* = {np.round(x_val, 4)}")
    print(f"Valor objetivo = {round(obj_val, 4)}")


--- Resolviendo con Γ = 1.0 ---
Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2679891
Academic license 2679891 - for non-commercial use only - registered to jc___@al.uloyola.es
Set parameter LogToConsole to value 0
Being solved by Gurobi...
Solution status: 2
Running time: 0.0150s
Solución x* = [0.9194 1.3792 0.     1.5324 2.7583]
Valor objetivo = 6.5893

--- Resolviendo con Γ = 2.0 ---
Set parameter LogToConsole to value 0
Being solved by Gurobi...
Solution status: 2
Running time: 0.0030s
Solución x* = [0.6914 1.0371 1.4815 1.1523 2.0741]
Valor objetivo = 6.4363

--- Resolviendo con Γ = 3.0 ---
Set parameter LogToConsole to value 0
Being solved by Gurobi...
Solution status: 2
Running time: 0.0030s
Solución x* = [0.     0.     0.     0.     6.3822]
Valor objetivo = 6.3822

--- Resolviendo con Γ = 4.0 ---
Set parameter LogToConsole to value 0
Being solved by Gurobi...
Solution status: 2
Running time: 0.0050s
Solución x* = [0.     0.     0.     0.   

# Intervalo [min max]

In [5]:
data = loadmat("Datos_Manufacturing_MinMax.mat")
A_MinMax = data['A_MinMax']
A_hat_MinMax = data['A_hat_MinMax']
b = data['b'].flatten()

c = np.array([1.0,1.0,1.0,1.0,1.0]) 
Gamma = 3
tolerance = 1e-6
max_iter = 30

In [7]:
# === Parámetros del problema ===
m, n = A_MinMax.shape

# Todos los coeficientes son inciertos
J = [list(range(n)) for _ in range(m)]

# === Probar con distintos valores de Gamma ===
for Gamma_value in [1.0, 2.0, 3.0, 4.0, 5.0]:
    print(f"\n--- Resolviendo con Γ = {Gamma_value} ---")

    Gamma = np.array([Gamma_value] * m)
    model = ro.Model()

    x = model.dvar(n)
    y = model.dvar(n)
    z = model.dvar(m)
    p = {}

    # Declarar variables auxiliares p_ij
    for i in range(m):
        for j in J[i]:
            p[i, j] = model.dvar()

    # Restricciones robustas
    for i in range(m):
        expr = sum(A_MinMax[i, j] * x[j] for j in range(n)) + Gamma[i] * z[i] + sum(p[i, j] for j in J[i])
        model.st(expr <= b[i])

        for j in J[i]:
            model.st(z[i] + p[i, j] >= A_hat_MinMax[i, j] * y[j])
            model.st(p[i, j] >= 0)

    # Relación y_j = |x_j|
    for j in range(n):
        model.st(x[j] <= y[j])
        model.st(-x[j] <= y[j])
        model.st(y[j] >= 0)

    # Dominio de x
    model.st(x >= 0)

    # Objetivo
    model.max(c @ x)

    # Resolver
    model.solve(grb)

    # Resultados
    x_val = x.get()
    obj_val = model.get()
    print(f"Solución x* = {np.round(x_val, 4)}")
    print(f"Valor objetivo = {round(obj_val, 4)}")


--- Resolviendo con Γ = 1.0 ---
Set parameter LogToConsole to value 0
Being solved by Gurobi...
Solution status: 2
Running time: 0.0030s
Solución x* = [1.1934 1.2054 1.1934 1.1815 1.2054]
Valor objetivo = 5.9791

--- Resolviendo con Γ = 2.0 ---
Set parameter LogToConsole to value 0
Being solved by Gurobi...
Solution status: 2
Running time: 0.0050s
Solución x* = [1.0614 1.0721 1.0614 1.0509 1.0721]
Valor objetivo = 5.318

--- Resolviendo con Γ = 3.0 ---
Set parameter LogToConsole to value 0
Being solved by Gurobi...
Solution status: 2
Running time: 0.0030s
Solución x* = [0.9557 0.9654 0.9557 0.9463 0.9654]
Valor objetivo = 4.7886

--- Resolviendo con Γ = 4.0 ---
Set parameter LogToConsole to value 0
Being solved by Gurobi...
Solution status: 2
Running time: 0.0030s
Solución x* = [0.8692 0.878  0.8692 0.8606 0.878 ]
Valor objetivo = 4.355

--- Resolviendo con Γ = 5.0 ---
Set parameter LogToConsole to value 0
Being solved by Gurobi...
Solution status: 2
Running time: 0.0030s
Solución x* 